In [1]:
# Загрузка данных
with open('/content/Viking.txt', 'r', encoding='utf-8') as file:
    script_lines = file.readlines()

In [2]:
# Очистка данных
dialogues = []
current_speaker = None
for line in script_lines:
    # Удаление ненужных символов
    cleaned_line = line.strip().replace('\u266A', '').replace('<', '').replace('>', '').replace('U+266A', '').strip()

    # Проверка является ли строка диалогом
    if ':' in cleaned_line:
        split_line = cleaned_line.split(':', 1)
        current_speaker = split_line[0].strip()
        dialogue = split_line[1].strip()
        if dialogue:
            dialogues.append({'speaker': current_speaker, 'dialogue': dialogue})
    elif current_speaker and cleaned_line:
        dialogues.append({'speaker': current_speaker, 'dialogue': cleaned_line})

In [3]:
dialogues[:50]

[{'speaker': 'Latvian warrior', 'dialogue': '(Attack yell)'},
 {'speaker': 'Latvian warrior', 'dialogue': '(Fighting grunts)'},
 {'speaker': 'Latvian warrior', 'dialogue': 'Agghhh!'},
 {'speaker': 'Latvian warrior', 'dialogue': 'Ungh!'},
 {'speaker': 'Latvian warrior', 'dialogue': 'Ohhh!'},
 {'speaker': 'Latvian warrior', 'dialogue': 'Agghhh!'},
 {'speaker': 'Latvian warrior', 'dialogue': 'Agh! Ugh!'},
 {'speaker': 'Latvian warrior', 'dialogue': 'Agghhhh!'},
 {'speaker': 'Latvian warrior', 'dialogue': '(Latvian warrior yells)'},
 {'speaker': 'Scandinavian warrior', 'dialogue': 'Ragnar!'},
 {'speaker': 'Scandinavian warrior', 'dialogue': 'Agggghhhh! (Weapons clank)'},
 {'speaker': 'Scandinavian warrior', 'dialogue': 'Ungh! Ungh!'},
 {'speaker': 'Scandinavian warrior',
  'dialogue': '(Hunting knife slices) Agh!'},
 {'speaker': 'Scandinavian warrior', 'dialogue': 'Ungh! Ungh!'},
 {'speaker': 'Scandinavian warrior', 'dialogue': '(Gurgling grunt)'},
 {'speaker': 'Scandinavian warrior', 'dia

In [4]:
# Отфильтруем строки
filtered_dialogues = [d for d in dialogues if not any(char in d['dialogue'] for char in '()') and len(d['dialogue'].split()) > 1]

In [5]:
filtered_dialogues[:25]

[{'speaker': 'Latvian warrior', 'dialogue': 'Agh! Ugh!'},
 {'speaker': 'Scandinavian warrior', 'dialogue': 'Ungh! Ungh!'},
 {'speaker': 'Scandinavian warrior', 'dialogue': 'Ungh! Ungh!'},
 {'speaker': 'Rollo', 'dialogue': 'More, give me more'},
 {'speaker': 'Rollo', 'dialogue': 'give me more'},
 {'speaker': 'Rollo', 'dialogue': 'if I had a heart I could love you'},
 {'speaker': 'Rollo', 'dialogue': 'if I had a voice I would sing'},
 {'speaker': 'Rollo', 'dialogue': 'after the night when I wake up'},
 {'speaker': 'Rollo', 'dialogue': "I'll see what tomorrow brings"},
 {'speaker': 'Rollo', 'dialogue': 'I... I... I...'},
 {'speaker': 'Rollo', 'dialogue': 'If I had a voice I would sing'},
 {'speaker': 'Rollo', 'dialogue': "Come on. Let's go home."},
 {'speaker': 'Lagertha', 'dialogue': 'What are you two doing?'},
 {'speaker': 'Lagertha',
  'dialogue': "I'm taking Bjorn to the thing tomorrow."},
 {'speaker': 'Lagertha', 'dialogue': "Not yet. He's not old enough."},
 {'speaker': 'Lagertha', 

In [6]:
# Посчитаем количество диалогов на одного говорящего
from collections import Counter
dialogue_counter = Counter([d['speaker'] for d in filtered_dialogues])

In [7]:
dialogue_counter.most_common(), filtered_dialogues[:10]

([('Ragnar', 1054),
  ('Man', 693),
  ('Floki', 693),
  ('Bjorn', 692),
  ('Ecbert', 670),
  ('Lothbrok', 611),
  ('Lagertha', 578),
  ('Rollo', 542),
  ('Athelstan', 495),
  ('Siggy', 467),
  ('All', 410),
  ('Aethelwulf', 339),
  ('Aslaug', 335),
  ('Seer', 282),
  ('Ubbe', 246),
  ('Helga', 232),
  ('Ivar', 221),
  ('King Horik', 182),
  ('Earl Haraldson', 181),
  ('Odo', 170),
  ('King Ecbert', 165),
  ('Athelstan voiceover', 165),
  ('Man 4', 163),
  ('Harald', 163),
  ('Woman', 159),
  ('Haraldson', 154),
  ('Einar', 150),
  ('Kwenthrith', 149),
  ('Man 2', 147),
  ('Soldier', 146),
  ('Neil', 134),
  ('Jarl Borg', 133),
  ('Roland', 130),
  ('Kalf', 119),
  ('Hvitserk', 117),
  ('KING HORIK', 107),
  ('Judith', 107),
  ('Gisla', 106),
  ('FLOKI', 105),
  ('RAGNAR', 101),
  ('Svein', 100),
  ('Torstein', 100),
  ('Captain', 100),
  ('Crowd', 97),
  ('MAN', 97),
  ('The Seer', 93),
  ('King Aelle', 85),
  ('TORSTEIN', 80),
  ('BJORN', 78),
  ('LAGERTHA', 77),
  ('SIGGY', 73),
  ('

In [8]:
!pip install -q transformers

In [9]:
!pip install -q torch

In [10]:
from sklearn.model_selection import train_test_split

dialogs = [d['dialogue'] for d in filtered_dialogues]
speakers = [d['speaker'] for d in filtered_dialogues]

# Разделение данных на обучающую и тестовую выборки
train_texts, val_texts, train_labels, val_labels = train_test_split(dialogs, speakers, test_size=.2)

In [11]:
# подключимся к аккаунту
from huggingface_hub import notebook_login
notebook_login()

In [12]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Загрузка предобученного токенизатора и модели
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(set(speakers)))

# from transformers import RobertaTokenizer, RobertaForSequenceClassification

# tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
# model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=len(set(speakers)))

# Токенизация текстов
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
# Создание словаря для маппинга имен персонажей в числовые идентификаторы
label_dict = {label: idx for idx, label in enumerate(set(speakers))}

# Преобразование строковых меток в числовые идентификаторы
train_labels = [label_dict[label] for label in train_labels]
val_labels = [label_dict[label] for label in val_labels]

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [15]:
# Преобразование данных в формат, подходящий для torch
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [16]:
train_dataset = Dataset(train_encodings, train_labels)
val_dataset = Dataset(val_encodings, val_labels)

In [17]:
from torch.utils.data import DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup

# Создание загрузчиков данных
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)

# Оптимизатор и функция потерь
optimizer = AdamW(model.parameters(), lr=5e-5)

# Общее количество шагов обучения
total_steps = len(train_loader) * 3

# Планировщик для изменения скорости обучения
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,
                                            num_training_steps=total_steps)

# Функция для обучения модели
def train():
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        scheduler.step()
    return total_loss / len(train_loader)

# Функция для оценки модели
def evaluate():
    model.eval()
    total_loss = 0
    for batch in val_loader:
        with torch.no_grad():
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            total_loss += outputs.loss.item()
    return total_loss / len(val_loader)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [18]:
# Обучение модели
for epoch in range(3):
    train()
    val_loss = evaluate()
    print(f'Epoch {epoch}, Val loss: {val_loss}')

Epoch 0, Val loss: 4.418389072418213
Epoch 1, Val loss: 4.360135707855225
Epoch 2, Val loss: 4.344935455322266


In [21]:
def predict(text, model, tokenizer):
    # Токенизация текста
    inputs = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors="pt").to(device)

    # Получение предсказания от модели
    with torch.no_grad():
        outputs = model(**inputs)

    # Получение индекса наиболее вероятного ответа
    probs = outputs.logits.softmax(dim=-1)
    predicted_label_idx = probs.argmax(dim=-1).item()

    # Преобразование индекса метки обратно в имя персонажа
    predicted_label = [key for key, value in label_dict.items() if value == predicted_label_idx][0]

    return predicted_label

In [26]:
# Пример использования:
text = "What are you two doing?"
predicted_character = predict(text, model, tokenizer)
print(f"Predicted character: {predicted_character}")

Predicted character: Ragnar
